In [4]:
'''
    概述：这个做成任务，每周或更久才更新一次。或者数据量大的变化时更新。不用太及时。
    做法是取出一些标题（不用所有，太久的太多或没用，自己把握取多少，速度能接受就行），然后切词，然后统计词频，然后取高频词，然后拼音，
    都准备好后，清空现有suggest，插入新的。（因为更新貌似更麻烦还不如重建）
    具体如下：

    scan all fgw_index items_of_topn_or_past_6month:
        get title and analyze to tokens #ares=es.indices.analyze(index='fgw',body=dict(text='政府投资及企业投资类铁路项目备案',analyzer='ik_synonym_analyzer'),filter_path="tokens.token")['tokens']
    count tokens
    take token if len() >1 and count>n,n can set 100 or 50.
    maybe need to skip stop words
    
    for token in selected_tokens:
        get pinyin tokens:ares=es.indices.analyze(index='fgw',body=dict(text=l['_source']['ruiso_suggest']['input'][0],analyzer='pinyin_analyzer'),filter_path="tokens.token")['tokens']
        add to items_to_update list,with token,pinyins,weight=count
    
    clear and insert items
'''

'\n    概述：这个做成任务，每周或更久才更新一次。或者数据量大的变化时更新。不用太及时。\n    做法是取出一些标题（不用所有，太久的太多或没用，自己把握取多少，速度能接受就行），然后切词，然后统计词频，然后取高频词，然后拼音，都准备好后，清空现有suggest，插入新的。（因为更新貌似更麻烦还不如重建）\n    \n\n    scan all fgw_index items_of_topn_or_past_6month:\n        get title and analyze to tokens #ares=es.indices.analyze(index=\'fgw\',body=dict(text=\'政府投资及企业投资类铁路项目备案\',analyzer=\'ik_synonym_analyzer\'),filter_path="tokens.token")[\'tokens\']\n    count tokens\n    take token if len() >1 and count>n,n can set 100 or 50.\n    maybe need to skip stop words\n    \n    for token in selected_tokens:\n        get pinyin tokens:ares=es.indices.analyze(index=\'fgw\',body=dict(text=l[\'_source\'][\'ruiso_suggest\'][\'input\'][0],analyzer=\'pinyin_analyzer\'),filter_path="tokens.token")[\'tokens\']\n        add to items_to_update list,with token,pinyins,weight=count\n    \n    clear and insert items\n'

In [2]:
import elasticsearch
from elasticsearch import Elasticsearch,helpers
es=Elasticsearch("192.168.71.20:9200")

In [1]:
"""
{
	"fgw_suggest": {
		"aliases": {},
		"mappings": {
			"properties": {
				"ruiso_suggest": {
					"type": "completion",
					"analyzer": "simple",
					"preserve_separators": true,
					"preserve_position_increments": true,
					"max_input_length": 15
				}
			}
		},
		"settings": {
			"index": {
				"routing": {
					"allocation": {
						"include": {
							"_tier_preference": "data_content"
						}
					}
				},
				"number_of_shards": "1",
				"provided_name": "fgw_suggest",
				"creation_date": "1666958173394",
				"number_of_replicas": "1",
				"uuid": "gzWOm1IjTzOswuBs1SF5uw",
				"version": {
					"created": "7160299"
				}
			}
		}
	}
}
"""

'\n{\n\t"fgw_suggest": {\n\t\t"aliases": {},\n\t\t"mappings": {\n\t\t\t"properties": {\n\t\t\t\t"ruiso_suggest": {\n\t\t\t\t\t"type": "completion",\n\t\t\t\t\t"analyzer": "simple",\n\t\t\t\t\t"preserve_separators": true,\n\t\t\t\t\t"preserve_position_increments": true,\n\t\t\t\t\t"max_input_length": 15\n\t\t\t\t}\n\t\t\t}\n\t\t},\n\t\t"settings": {\n\t\t\t"index": {\n\t\t\t\t"routing": {\n\t\t\t\t\t"allocation": {\n\t\t\t\t\t\t"include": {\n\t\t\t\t\t\t\t"_tier_preference": "data_content"\n\t\t\t\t\t\t}\n\t\t\t\t\t}\n\t\t\t\t},\n\t\t\t\t"number_of_shards": "1",\n\t\t\t\t"provided_name": "fgw_suggest",\n\t\t\t\t"creation_date": "1666958173394",\n\t\t\t\t"number_of_replicas": "1",\n\t\t\t\t"uuid": "gzWOm1IjTzOswuBs1SF5uw",\n\t\t\t\t"version": {\n\t\t\t\t\t"created": "7160299"\n\t\t\t\t}\n\t\t\t}\n\t\t}\n\t}\n}\n'

In [7]:
total_items=es.count(index='fgw_suggest')['count']

In [25]:
from tqdm import tqdm

In [16]:
import time

In [13]:
ares=es.indices.analyze(index='fgw',body=dict(text='中国人民银行',analyzer='ik_synonym_search_analyzer'))#,filter_path="tokens.token")['tokens']
ares

{'tokens': [{'token': '中国人民银行',
   'start_offset': 0,
   'end_offset': 6,
   'type': 'CN_WORD',
   'position': 0}]}

In [36]:
ares=es.indices.analyze(index='fgw',body=dict(text='G-20',analyzer='pinyin_analyzer'))#,filter_path="tokens.token")['tokens']
ares

{'tokens': [{'token': 'g',
   'start_offset': 0,
   'end_offset': 0,
   'type': 'word',
   'position': 0},
  {'token': 'g-20',
   'start_offset': 0,
   'end_offset': 0,
   'type': 'word',
   'position': 0},
  {'token': 'g20',
   'start_offset': 0,
   'end_offset': 0,
   'type': 'word',
   'position': 0},
  {'token': '20',
   'start_offset': 0,
   'end_offset': 0,
   'type': 'word',
   'position': 1}]}

In [7]:
import re

In [53]:
from collections import Counter
token_freqs=Counter()

In [54]:
token_freqs.update(['这里','没用','自己','拼音'])
token_freqs.update(['这里','没用','自己','拼音'])
token_freqs.update(['这里','没用','自己','拼音'])
token_freqs.update(['这里','没用','自己','拼音'])

In [55]:
token_freqs.most_common(100)

[('这里', 4), ('没用', 4), ('自己', 4), ('拼音', 4)]

In [49]:
'''原理：
这里没用自己的拼音，而是用了es的拼索引的拼音功能。这里以前已经有了fgw_suggest，
'''
import re
_re_py=re.compile('[a-zA-Z ]+')

def es_analyze_zh_cut(raw_text):
    ares=es.indices.analyze(index='fgw',body=dict(text=raw_text,analyzer='ik_synonym_search_analyzer'),filter_path="tokens.token")['tokens']
    ares=[e['token'] for e in ares]
    return ares

def es_analyze_suggest_and_pinyings(raw_token):
    ares=es.indices.analyze(index='fgw',body=dict(text=raw_token,analyzer='pinyin_analyzer'),filter_path="tokens.token")['tokens']
    ares=[e['token'] for e in ares if e['token'] !=raw_token.lower()]
    all_compls=[raw_token]
    if len(ares)>1:
        #https://github.com/medcl/elasticsearch-analysis-pinyin 这个顺需和返回得token 和这些有关。0是第一个拼音，1是汉字,2是首字母，3以后是其他字母
        #这个或许应把 mapping 中的pinyin_analyzer keep_first_letter 关掉，然后不要首字母，然后自己用re去掉汉字，然后根据 position 排序后自己拼接，那样更安全
        all_compls.append(ares.pop(1))
        all_compls.append(''.join(ares))
        all_compls.append(' '.join(ares))
        
        #all_compls+=[ares[2],''.join(ares[0:1]+ares[3:]),' '.join(ares[0:1]+ares[3:])]
        return all_compls
        
    #return None# 没有拼音的一般都是有任何汉字的。



In [ ]:
from collections import Counter
token_freqs=Counter()
total_items=es.count(index='fgw')['count']
for li,l in tqdm(enumerate(helpers.scan(client=es,index='fgw',query=dict(_source=["title"],query=dict(term=dict(scene="news"))))),total=total_items):
    l=l['_source'].get('title')
    ts=es_analyze_zh_cut(l)
    token_freqs.update(ts)

In [50]:
newitems=[]
most_commons=[(a,b) for a,b in token_freqs.most_common(10000) if b>20 and len(a)>1]

for raw_token,token_weight in tqdm(most_commons):
    all_compls=es_analyze_suggest_and_pinyings(raw_token)
    if all_compls:
        newbody={'_op_type': 'index','_index': 'fgw_suggest','doc':{'ruiso_suggest':{'input':all_compls,'weight':token_weight}}}
        newitems.append(newbody)
    

  0%|                                                                         | 0/802 [00:00<?, ?it/s]C:\ProgramData\Miniconda3\lib\site-packages\elasticsearch\connection\base.py:177: ElasticsearchDeprecationWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.16/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchDeprecationWarning)
100%|██████████████████████████████████████████████████████████████| 802/802 [00:01<00:00, 651.14it/s]


In [ ]:
most_commons

In [29]:
'''原理：
这里没用自己的拼音，而是用了es的拼音索引的拼音功能。这里以前已经有了fgw_suggest，
'''

newitems=[]
for li,sl in tqdm(enumerate(helpers.scan(client=es,index='fgw_suggest')),total=total_items):
    l=sl
    ares=es.indices.analyze(index='fgw',body=dict(text=l['_source']['ruiso_suggest']['input'][0],analyzer='pinyin_analyzer'),filter_path="tokens.token")['tokens']
    ares=[e['token'] for e in ares]
    if not ares or len(ares)<2:
        continue
    #print(ares,l)
    #break
    all_compls=[ares[1]]
    if len(ares)>2:
        all_compls+=[ares[2],''.join(ares[0:1]+ares[3:]),' '.join(ares[0:1]+ares[3:])]

    newbody={'_op_type': 'update','_index': 'fgw_suggest','_id': l['_id'],'doc':{'ruiso_suggest':{'input':all_compls,'weight':l['_source']['ruiso_suggest']['weight']}}}
    newitems.append(newbody)
        

100%|███████████████████████████████████████████████████████████████████████████| 62828/62828 [02:19<00:00, 450.99it/s]


In [34]:
sl

{'_index': 'fgw_suggest',
 '_type': '_doc',
 '_id': 'vOl8HoQByB9vxMP5ePOY',
 '_score': None,
 '_source': {'ruiso_suggest': {'input': ['中共中央党'], 'weight': 4}},
 'sort': [62827]}

In [31]:
newitems[:3]

[{'_op_type': 'update',
  '_index': 'fgw_suggest',
  '_id': 'Ueh2HoQByB9vxMP54f6a',
  '_source': {'ruiso_suggest': {'input': ['发展', 'fz', 'fazhan', 'fa zhan'],
    'weight': 17554}}},
 {'_op_type': 'update',
  '_index': 'fgw_suggest',
  '_id': 'Uuh2HoQByB9vxMP54v5Y',
  '_source': {'ruiso_suggest': {'input': ['企业', 'qy', 'qiye', 'qi ye'],
    'weight': 9718}}},
 {'_op_type': 'update',
  '_index': 'fgw_suggest',
  '_id': 'U-h2HoQByB9vxMP54v5p',
  '_source': {'ruiso_suggest': {'input': ['工作', 'gz', 'gongzuo', 'gong zuo'],
    'weight': 7937}}}]

In [45]:
def update_suggests(items):
    global exx
    try:
        helpers.bulk(client = es,actions = items)
    except Exception as ex:
        exx=ex
        print(ex)
        raise ex


In [ ]:
        gen = _process_bulk_chunk_error(
            error=e,
            bulk_data=bulk_data,
            raise_on_exception=raise_on_exception,
            raise_on_error=raise_on_error,
        )

In [49]:
exx

elasticsearch.exceptions.RequestError(400,
                                      'x_content_parse_exception',
                                      {'error': {'root_cause': [{'type': 'x_content_parse_exception',
                                          'reason': '[1:2] [UpdateRequest] unknown field [ruiso_suggest]'}],
                                        'type': 'x_content_parse_exception',
                                        'reason': '[1:2] [UpdateRequest] unknown field [ruiso_suggest]'},
                                       'status': 400})

In [51]:
update_suggests([{'_op_type': 'update',
  '_index': 'fgw_suggest',
  '_id': 'Ueh2HoQByB9vxMP54f6a',
  'doc': {'ruiso_suggest': {'input': ['发展', 'fz', 'fazhan', 'fa zhan'],
    'weight': 17554}}}])

C:\ProgramData\Miniconda3\lib\site-packages\elasticsearch\connection\base.py:177: ElasticsearchDeprecationWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.16/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchDeprecationWarning)


In [52]:
for i in tqdm(range(0,len(newitems),100)):
    x=[{(k if k!="_source" else "doc"):v for k,v in item.items()} for item in newitems[i:i+100]]
    update_suggests(x)


100%|████████████████████████████████████████████████████████████████████████████████| 623/623 [00:19<00:00, 32.32it/s]


In [54]:
newitems[0],helpers.expand_action(newitems[0])

({'_op_type': 'update',
  '_index': 'fgw_suggest',
  '_id': 'Ueh2HoQByB9vxMP54f6a',
  '_source': {'ruiso_suggest': {'input': ['发展', 'fz', 'fazhan', 'fa zhan'],
    'weight': 17554}}},
 ({'update': {'_id': 'Ueh2HoQByB9vxMP54f6a', '_index': 'fgw_suggest'}},
  {'ruiso_suggest': {'input': ['发展', 'fz', 'fazhan', 'fa zhan'],
    'weight': 17554}}))

In [24]:
newitems[:10]

[{'_op_type': 'update',
  '_index': 'fgw_suggest',
  '_id': 'Ueh2HoQByB9vxMP54f6a',
  '_source': {'ruiso_suggest': {'input': ['发展', 'fz', 'fazhan', 'fa zhan'],
    'weight': 17554}}},
 {'_op_type': 'update',
  '_index': 'fgw_suggest',
  '_id': 'Uuh2HoQByB9vxMP54v5Y',
  '_source': {'ruiso_suggest': {'input': ['企业', 'qy', 'qiye', 'qi ye'],
    'weight': 9718}}},
 {'_op_type': 'update',
  '_index': 'fgw_suggest',
  '_id': 'U-h2HoQByB9vxMP54v5p',
  '_source': {'ruiso_suggest': {'input': ['工作', 'gz', 'gongzuo', 'gong zuo'],
    'weight': 7937}}},
 {'_op_type': 'update',
  '_index': 'fgw_suggest',
  '_id': 'VOh2HoQByB9vxMP54v53',
  '_source': {'ruiso_suggest': {'input': ['年', 'n', 'nian', 'nian'],
    'weight': 7573}}},
 {'_op_type': 'update',
  '_index': 'fgw_suggest',
  '_id': 'Veh2HoQByB9vxMP54v6E',
  '_source': {'ruiso_suggest': {'input': ['月', 'y', 'yue', 'yue'],
    'weight': 7074}}},
 {'_op_type': 'update',
  '_index': 'fgw_suggest',
  '_id': 'Vuh2HoQByB9vxMP54v6V',
  '_source': {'rui